In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
model_name = "kaiyuy/leandojo-lean3-retriever-tacgen-byt5-small"
device = 'cuda'
num_retrieved = 2

In [5]:
from reprover.retrieval.datamodule import RetrievalDataset
from reprover.common import Corpus

corpus_path = '../data/leandojo_benchmark_4/corpus.jsonl'
premises_paths_list = ["../data/leandojo_benchmark_4/random/train.json",
                       "../data/leandojo_benchmark_4/random/val.json",
                       "../data/leandojo_benchmark_4/random/test.json"]

corpus = Corpus(corpus_path)

retrieval_dataset = RetrievalDataset(
    premises_paths_list,
    True,
    corpus,
    num_negatives=1,
    num_in_file_negatives=1,
    max_seq_len=None,
    context_tokenizer=None,
    premise_tokenizer=None,
    is_train=True,
)

example = retrieval_dataset.data[0]
ctx = example['context']


inputs = dict(
    state=[ctx.state],
    file_name=[ctx.path],
    theorem_full_name=[ctx.theorem_full_name],
    theorem_pos=[ctx.theorem_pos]
)

2024-03-16 17:11:40.051 | INFO     | reprover.retrieval.datamodule:_load_data:54 - Loading data from ../data/leandojo_benchmark_4/random/train.json
  0%|          | 0/98514 [00:00<?, ?it/s]2024-03-16 17:11:51.393 | WARNING  | reprover.common:get_all_pos_premises:339 - Cannot locate premise: {'full_name': 'neg_lt_zero', 'def_path': 'Mathlib/Algebra/Order/Group/Defs.lean', 'def_pos': [601, 24], 'def_end_pos': [601, 35]}
2024-03-16 17:11:51.432 | WARNING  | reprover.common:get_all_pos_premises:339 - Cannot locate premise: {'full_name': 'AddMonoidHomClass.isometry_of_norm', 'def_path': 'Mathlib/Analysis/Normed/Group/Basic.lean', 'def_pos': [894, 12], 'def_end_pos': [894, 23]}
2024-03-16 17:11:51.441 | WARNING  | reprover.common:get_all_pos_premises:339 - Cannot locate premise: {'full_name': 'add_nonneg', 'def_path': 'Mathlib/Algebra/Order/Monoid/Lemmas.lean', 'def_pos': [1092, 24], 'def_end_pos': [1092, 34]}
2024-03-16 17:11:51.445 | WARNING  | reprover.common:get_all_pos_premises:339 - Ca

In [6]:
from reprover.retrieval.model import PremiseRetriever

retriever = PremiseRetriever(
    model_name=model_name,
    lr=3e-4,
    warmup_steps=1000,
    max_seq_len=1024,
    num_retrieved=num_retrieved
).to(device)
retriever.load_corpus(corpus_path)
retriever.corpus.all_premises = retriever.corpus.all_premises[:100]

retriever.retrieve(
    **inputs,
    k=2,
    reindex_batch_size=32
)

2024-03-16 17:12:19.391 | INFO     | reprover.retrieval.model:reindex_corpus:128 - Re-indexing the retrieval corpus
100%|██████████| 4/4 [00:01<00:00,  2.65it/s]


([[Premise(path='lake-packages/lean4/src/lean/Init/Prelude.lean', full_name='NonemptyType.type', code='abbrev NonemptyType.type (type : NonemptyType.{u}) : Type u :=\n  type.val'),
   Premise(path='lake-packages/lean4/src/lean/Init/Prelude.lean', full_name='Classical.ofNonempty', code='noncomputable def Classical.ofNonempty {α : Sort u} [Nonempty α] : α :=\n  Classical.choice inferInstance')]],
 [[0.6735717058181763, 0.6651866436004639]])

In [ ]:
from reprover.retrieval.colbert.model import ColBERTPremiseRetriever

collection_path = '../data/leandojo_benchmark_4/novel_premises/colbert_collection.tsv'

retriever = ColBERTPremiseRetriever(
    index_path=index_path,
    checkpoint_path_or_name=,
    collection=collection_path,
    num_retrieved=num_retrieved,
)

retriever.load_corpus(corpus_path)
retriever.retrieve(
    **inputs,
    k=2,
    reindex_batch_size=32
)